# Homework 6: Part I

1. Go get data from kaggle.com and do a ***(Univariate) Bayesian Logistic Regression*** analysis

2. Adjust the code below to specify that the outcomes have a Bernoulli distribution and use a ***probit*** or ***logit*** to correctly paramterize the predicted values of the observed outcomes

```python
import pymc as pm; import numpy as np
n,p=100,10; X,y=np.zeros((n,p)),np.ones((n,1))
# Replace this made up data with your data set from kaggle...
with pm.Model() as MLR:

# > Indented block


    betas = pm.MvNormal('betas', mu=np.zeros((p,1)), cov=np.eye(p), shape=(p,1))
    sigma = pm.TruncatedNormal('sigma', mu=1, sigma=1, lower=0) # half normal
    y = pm.Normal('y', mu=pm.math.dot(X, betas), sigma=sigma, observed=y)

with MLR:
    idata = pm.sample()
```    

3. Choose ***prior*** that are sensible for your specification

4. [Optional] Assess the performance of the MCMC and any issues or warnings in the [standard manner](https://www.pymc.io/projects/docs/en/stable/learn/core_notebooks/pymc_overview.html#pymc-overview)

4. [Optional] Go get data from kaggle.com and do a ***Multivariate Bayesian Logistic Regression*** analysis


In [3]:
import pymc as pm; import numpy as np
n,p=80,5; X,y=np.zeros((n,p)),np.ones((n,1))
with pm.Model() as logistic_model:
    betas = pm.MvNormal('betas', mu=np.zeros((p,1)), cov=np.eye(p), shape=(p,1))
    linear_comb = pm.math.dot(X, betas)
    p = pm.math.sigmoid(linear_comb)
    y_obs = pm.Bernoulli('y_obs', p=p, observed=y)
with logistic_model:
    idata = pm.sample()


# Homework 6: Part II<br>Robust regression: [scale mixtures (of normals)](https://www.johndcook.com/t_normal_mixture.pdf)

$$\int \frac{w\lambda_i}{\sqrt{2\pi}}e^{-\frac{1}{2}\left(w^{-2}\lambda_i^{-2}(y_i-\mu)^2\right)} {\frac {\frac{\nu}{2}^{\frac{\nu}{2}}}{\Gamma (\frac{\nu}{2})}}\lambda_i^{\nu-\frac{1}{2}}e^{-\frac{\nu}{2}\lambda_i^{-2}} d\lambda_i = \frac{\Gamma\left(\frac{\nu+1}{2}\right)}{\Gamma (\frac{\nu}{2})\sqrt{\pi \nu w^2}}\left(1 + \frac{1}{\nu}\left(\frac{y_i-\mu}{w}\right)^2 \right)^{-\frac{\nu+1}{2}}$$

$$\begin{array}{rcl}
y_i|\lambda_i & \sim & \mathcal{N}(X\beta, \sigma^2 = w^2 \lambda_i^2)\\
\lambda_i^{-2} & \sim & \text{Gamma}(\alpha=\nu/2, \beta=\nu/2)\end{array} \quad \Longrightarrow \quad y_i \sim t_\nu(\mu, w^2)$$

1. Return to your kaggle.com regression data set; or, find another data set; and use the above specification to perform a robust regression analysis in `PyMC`

2. Use the posterior distributions of the $\lambda_i$'s to identify "outlier" (and potentially "influential") data points

3. [Optional] Assess the performance of the MCMC and any issues or warnings in the [standard manner](https://www.pymc.io/projects/docs/en/stable/learn/core_notebooks/pymc_overview.html#pymc-overview)
4. [Optional] Perform ***Multiple Linear Regression*** diagnostics... residual plots, etc.




In [5]:
import pymc as pm
import numpy as np
import arviz as az

np.random.seed(36)
n_samples = 80
X = np.random.normal(size=(n_samples, 1))
true_beta = 2.5
mu = X * true_beta
w = 1
nu = 3

y = np.random.normal(mu.flatten(), scale=w)

with pm.Model() as robust_model_with_lambda:
    beta = pm.Normal('beta', mu=0, sigma=10)
    nu = pm.Exponential('nu', 1/29) + 1
    lambda_i = pm.Gamma('lambda', alpha=nu/2, beta=nu/2, shape=n_samples)
    sigma_i = pm.Deterministic('sigma_i', 1 / (lambda_i))

    likelihood = pm.Normal('y', mu=X[:,0]*beta, sigma=sigma_i, observed=y)

    trace = pm.sample(1000, target_accept=0.9, return_inferencedata=True)

lambda_posterior = trace.posterior['lambda'].values
lambda_mean = np.mean(lambda_posterior, axis=(0, 1))
outlier_threshold = np.quantile(lambda_mean, 0.05)
outliers = np.where(lambda_mean < outlier_threshold)[0]
print(f"Potential outliers: {outliers}")


Potential outliers: [28 36 60 75]
